In [32]:
%%writefile test_units.py

from hypothesis import strategies as st, assume, given
from pytest import fixture, raises
from unittest import TestCase, skip

from store.store import TripleStore, E, Predicate, Query, QuerySet

@given(st.text())
def test_E(url):
    # hypothesis testing on name should be possible, but it must be an identifier
    e1 = E()
    e2 = eval(repr(e1))
    # Equality is determined by their UUID, which must be the same now.
    assert e1 == e2
    # However, they can't be the same object.
    assert e1 is not e2
    # On the other hand, an entity created on its own must have a new UUID.
    e3 = E()
    assert e1 != e3
    # The id argument must be a valid UUID.
    with raises(AttributeError, message="'int' object has no attribute 'replace'"):
        E(id_=2)
    # Let's use all arguments.
    e4 = E(name='test', id_='b19102c6-20b8-4afb-8520-ef910b1dc93b',url=url)
    assert str(e4) == "test"
    assert repr(e4) == f"E(name='test', id_='b19102c6-20b8-4afb-8520-ef910b1dc93b', url='{url}')"
    # repr code often is quite noisy, so we need to check that
    assert repr(e1) == f"E(id_='{e1.id}')"
    test = E(name="test")
    assert str(test) == "test"
    assert repr(test) == f"E(name='test', id_='{test.id}')"
    
def test_Predicate():
    class Test(Predicate):
        def validate(self, value):
            return isinstance(value, str)
        
    test = Test()
    assert test.validate("test")
    assert test.name == "Test"
    


Overwriting test_units.py


In [28]:
%%writefile test_functionality.py

from pytest import fixture, raises
from unittest.mock import Mock
from unittest import skip

from store.store import TripleStore, E, Predicate, Query, Triple

@fixture
def body():
    return TripleStore()

@fixture
def name():
    class Name(Predicate):
        pass
    return Name()

@fixture
def is_a():
    class Is_a(Predicate):
        pass
    return Is_a()

@fixture
def destroyed():
    class Destroyed(Predicate):
        pass
    return Destroyed()

@fixture
def has():
    class Has(Predicate):
        pass
    return Has()
    
def test_bodily_functions(body, name, is_a):
    # Let's make a new entity in body with the name "head".
    e = body.create_subjects_with({name:["head"]})[0]
    # Indexing body with the entity as single argument should return a dict with its attributes.
    assert body[e] == {name: {"head"}}
    # body.get should return a single entity, not an iterator.
    assert body.get({name:"head"}) == e
    # body.get_all should return a set
    assert body.get_all({name: "head"}) == {e}
    # Let's see if this works with an enitity made outside.
    e2 = E()
    # We should be able to use the dict of attributes of a single index as argument to set_all.
    body.set_all(subjects=[e2], predobjects=body[e])
    # body.get_last_added should return the entity inserted last into the store.
    assert body.get_last_added() == e2
    # body[::] should return a set with all triples in the store, the iterator over body should produce the same.
    assert body[::] == {(e, name, "head"), (e2, name, "head")} == {x for x in body}
    # Using the index-assignment pattern should work for 1 or many entities as subjects, a given pred and an object.
    body[body[:name:"head"]:is_a] = "part"
    # body.get_all with a dict should produce the same result as using slices and ANDing the produced sets.
    assert body.get_all({is_a:"part", name:"head"}) == {e, e2} == body[:name:"head"] & body[:is_a:"part"]
    

def test_Triple_as_subject(body, has, destroyed):
    hand = E(name="hand")
    ring = E(name="ring")
    body[hand:has] = ring
    assert Triple(hand, has, ring) in body
    body[Triple(hand, has, ring):destroyed] = True

Overwriting test_functionality.py
